In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import thresholding as thresh
from os import path

In [2]:
def click_and_crop(event, x, y, flags, param):
    global refPt, cropping
    if event == cv2.EVENT_LBUTTONDOWN:
        refPt = [(x, y)]
        cropping = True
    elif event == cv2.EVENT_LBUTTONUP:
        refPt.append((x, y))
        cropping = False
        cv2.rectangle(frame, refPt[0], refPt[1], (0, 0, 255), 2)
        cv2.imshow("image", frame)

In [42]:
refPt = []
cropping = False
roi_exist = False

cap = cv2.VideoCapture(0)

cv2.namedWindow("image")
cv2.setMouseCallback("image", click_and_crop)
while True:
    ret, frame = cap.read()
    width = int(cap.get(3))
    height = int(cap.get(4))
    if roi_exist == True:
        image = np.zeros(frame.shape, np.uint8)
        frame_ = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        roi_ = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
        img_para = thresh.segment_parametric(frame_,roi_)
        img_thresh = thresh.segment_thresholding(frame_,roi_)
        
        #denoising
        #img_thresh = cv2.fastNlMeansDenoising(img_thresh, None, 10, 7, 21) 
        #img_para = cv2.fastNlMeansDenoising(img_para, None, 10, 7, 21)

        #converting live-video feed to 4 parts
        smaller_frame = cv2.resize(frame, (0,0), fx = 0.5, fy = 0.5)
        smaller_para = cv2.resize(img_para, (0,0), fx = 0.5, fy = 0.5)
        smaller_thresh = cv2.resize(img_thresh, (0,0), fx = 0.5, fy = 0.5)
        smaller_still = cv2.resize(still, (0,0), fx = 0.5, fy = 0.5)
        
        image[:height//2, :width//2] = smaller_frame
        image[height//2:, :width//2] = smaller_still
        image[:height//2, width//2:] = np.stack((smaller_thresh, smaller_thresh, smaller_thresh), axis=2)
        image[height//2:, width//2:] = np.stack((smaller_para, smaller_para, smaller_para), axis=2)
        
        font = cv2.FONT_HERSHEY_SIMPLEX
        image = cv2.putText(image, 'Thresholding', (width//2+10, height//2 - 10), font, 1, (255, 0, 0), 2, cv2.LINE_AA)
        image = cv2.putText(image, 'Parametric Segmentation', (width//2+10, height - 10), font, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.imshow("image", image)
    else:
#         plt.imshow(frame)
#         plt.show()
        cv2.imshow("image", frame)
    
    key = cv2.waitKey(1) & 0xFF
    if len(refPt) == 2 and roi_exist == False:
        image = frame.copy()
        still = frame.copy()
        roi_exist = True
        roi = image[refPt[0][1]:refPt[1][1], refPt[0][0]:refPt[1][0]]
        #cv2.imshow("ROI", roi)
        cv2.imwrite('roi.png', roi)
        #cv2.waitKey(1)
    if key == ord("r"):
        refPt = []
        roi_exist = False
    elif key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

In [12]:
import numpy as np
import cv2 as cv
import argparse

#cap = cv.VideoCapture(0)
cap = cv2.VideoCapture('traffic.mp4')
# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100, qualityLevel = 0.3, minDistance = 7, blockSize = 7 )
lk_params = dict( winSize  = (15, 15), maxLevel = 2, criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))
color = np.random.randint(0, 255, (100, 3))
ret, old_frame = cap.read()
old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
p0 = cv.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
mask = np.zeros_like(old_frame)
while(1):
    ret, frame = cap.read()
    if not ret:
        print('No frames grabbed!')
        break
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    if p1 is not None:
        good_new = p1[st==1]
        good_old = p0[st==1]
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
        frame = cv.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)
    img = cv.add(frame, mask)
    cv.imshow('frame', img)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)
cv.destroyAllWindows()

In [14]:
cap = cv.VideoCapture('traffic.mp4')
ret, frame1 = cap.read()
prvs = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255
while(1):
    ret, frame2 = cap.read()
    if not ret:
        print('No frames grabbed!')
        break
    next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang*180/np.pi/2
    hsv[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
    cv.imshow('frame2', bgr)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
    elif k == ord('s'):
        cv.imwrite('opticalfb.png', frame2)
        cv.imwrite('opticalhsv.png', bgr)
    prvs = next
cv.destroyAllWindows()

In [17]:
import cv2

# Load the cascade
#face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# To capture video from webcam. 
cap = cv2.VideoCapture(0)
# To use a video file as input 
# cap = cv2.VideoCapture('filename.mp4')

while True:
    # Read the frame
    _, img = cap.read()
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Detect the faces
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    # Draw the rectangle around each face
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
    # Display
    cv2.imshow('img', img)
    # Stop if escape key is pressed
    k = cv2.waitKey(30) & 0xff
    if k==27:
        break
# Release the VideoCapture object
cap.release()